# Deep Memory Acquisition

The Deep Memory Acquisition represents another way to capture data with Red Pitaya. The differences between the standard SCPI/API data acquisition and the Deep Memory Acquisition (DMA) are the following:
- The data buffer is not limited to 16384 samples but can be specified by the user.
- The data is streamed directly into the DDR memory of the Red Pitaya
- The DDR is shared between Linux OS and DMA, so we recommend leaving a minimum of 100 MB of RAM to the Linux OS.
- The data can be acquired at full speed (125 MSPs) and is not limited like in the streaming application.
- The downside is that it might take a while to transfer the data out of the DDR once it is captured, especially if the data buffer is big.
- The DMA can work in parallel with standard SCPI acquisition.

The Deep Memory Acquisition allows you to set a buffer of any size (The buffer must be a multiple of 64 bytes) for capturing data from the ADC. The DMA relies on the AXI protocol. Consequently, the DMA functions are named after it.

More information regarding the Deep Memory Acquisition and how to change the buffer size is available [here](https://redpitaya.readthedocs.io/en/latest/appsFeatures/remoteControl/deepMemoryAcquisition.html#deep-memory-acquisition-dma).

## Libraries and FPGA image

In [ ]:
import time
import numpy as np
from matplotlib import pyplot as plt
from rp_overlay import overlay
import rp

fpga = overlay()
rp.rp_Init()

## Marcos
Throughout this tutorial we will mention macros multiple times. Here is a complete list of macros that will come in handy when customising this notebook. The marcos are a part of the **rp** library.

**GENERATION**

- **Waveforms** - RP_WAVEFORM_SINE, RP_WAVEFORM_SQUARE, RP_WAVEFORM_TRIANGLE, RP_WAVEFORM_RAMP_UP, RP_WAVEFORM_RAMP_DOWN, RP_WAVEFORM_DC, RP_WAVEFORM_PWM, RP_WAVEFORM_ARBITRARY, RP_WAVEFORM_DC_NEG, RP_WAVEFORM_SWEEP
- **Generator modes** - RP_GEN_MODE_CONTINUOUS, RP_GEN_MODE_BURST
- **Sweep direction** - RP_GEN_SWEEP_DIR_NORMAL, RP_GEN_SWEEP_DIR_UP_DOWN
- **Sweep mode** - RP_GEN_SWEEP_MODE_LINEAR, RP_GEN_SWEEP_MODE_LOG
- **Generator trigger source** - RP_GEN_TRIG_SRC_INTERNAL, RP_GEN_TRIG_SRC_EXT_PE, RP_GEN_TRIG_SRC_EXT_NE
- **Generator triggers** - RP_T_CH_1, RP_T_CH_2, RP_T_CH_EXT
- **Rise and fall times** - RISE_FALL_MIN_RATIO, RISE_FALL_MAX_RATIO

**ACQUISITION**

- **Decimation** - RP_DEC_1, RP_DEC_2, RP_DEC_4, RP_DEC_8, RP_DEC_16, RP_DEC_32, RP_DEC_64, RP_DEC_128, RP_DEC_256, RP_DEC_512, RP_DEC_1024, RP_DEC_2048, RP_DEC_4096, RP_DEC_8192, RP_DEC_16384, RP_DEC_32768, RP_DEC_65536 
- **Acquisition trigger** - RP_TRIG_SRC_DISABLED, RP_TRIG_SRC_NOW, RP_TRIG_SRC_CHA_PE, RP_TRIG_SRC_CHA_NE, RP_TRIG_SRC_CHB_PE, RP_TRIG_SRC_CHB_NE, RP_TRIG_SRC_EXT_PE, RP_TRIG_SRC_EXT_NE, RP_TRIG_SRC_AWG_PE, RP_TRIG_SRC_AWG_NE
- **Acquisition trigger state** - RP_TRIG_STATE_TRIGGERED, RP_TRIG_STATE_WAITING
- **Buffer size** - ADC_BUFFER_SIZE, DAC_BUFFER_SIZE
- **Fast analog channels** - RP_CH_1, RP_CH_2

SIGNALlab 250-12 only:
- **Input coupling** - RP_DC, RP_AC
- **Generator gain** - RP_GAIN_1X, RP_GAIN_5X

STEMlab 125-14 4-Input only:
- **Fast analog channels** - RP_CH_3, RP_CH_4
- **Acquisition trigger** - RP_TRIG_SRC_CHC_PE, RP_TRIG_SRC_CHC_NE, RP_TRIG_SRC_CHD_PE, RP_TRIG_SRC_CHD_NE

## Deep Memory Acquisition
This example shows how to capture data into two 1024-byte buffers, with comments on how to increase it to two 128 MB buffers.

In [ ]:
## size in samples 16Bit
DATA_SIZE = 1024          # ((1024 * 1024 * 128) / 2)        ## for 128 MB ##
READ_DATA_SIZE = 1024     # (1024 * 256)                     ## for 128 MB ##

dec = rp.RP_DEC_1
trig_lvl = 0.2

x = np.arange(1, READ_DATA_SIZE+1, 1)

data1 = np.zeros(READ_DATA_SIZE)
data2 = np.zeros(READ_DATA_SIZE)

Note that decimation is not limited to powers of 2, but can be chosen from the following array (1, 2, 4, 8, 16, 17, 18, 19, ..., 65535, 65536).

In [ ]:
### Setting up DMA ###
# Get Memory region
memoryRegion = rp.rp_AcqAxiGetMemoryRegion()
g_adc_axi_start = memoryRegion[1]       # The first address in the Deep Memory Acquisition region
g_adc_axi_size = memoryRegion[2]        # Size of the DMA region in samples
print(f"Reserved memory Start: {g_adc_axi_start:x} Size: {g_adc_axi_size:x}\n")

# Set decimation
rp.rp_AcqAxiSetDecimationFactor(dec)

# Set trigger delay for both channels
rp.rp_AcqAxiSetTriggerDelay(rp.RP_CH_1, DATA_SIZE)
rp.rp_AcqAxiSetTriggerDelay(rp.RP_CH_2, DATA_SIZE)

# Set-up the Channel 1 and channel 2 buffers to each work with half the available memory space.
rp.rp_AcqAxiSetBufferSamples(rp.RP_CH_1, g_adc_axi_start, DATA_SIZE)
rp.rp_AcqAxiSetBufferSamples(rp.RP_CH_2, g_adc_axi_start + int(g_adc_axi_size/2), DATA_SIZE)

# Enable DMA on both channels
rp.rp_AcqAxiEnable(rp.RP_CH_1, True)
print("Enable CHA\n")
rp.rp_AcqAxiEnable(rp.RP_CH_2, True)
print("Enable CHB\n")

# Specify the acquisition trigger
rp.rp_AcqSetTriggerLevel(rp.RP_T_CH_1, trig_lvl)


### Acquisition ###
# Start the DMA acquisition
rp.rp_AcqStart()
print("ACQ Started\n")

# Specify trigger source
rp.rp_AcqSetTriggerSrc(rp.RP_TRIG_SRC_CHA_PE)
state = rp.RP_TRIG_STATE_TRIGGERED

# Wait for the triggering moment
while 1:
    state = rp.rp_AcqGetTriggerState()[1]
    if state == rp.RP_TRIG_STATE_TRIGGERED:
        print("Triggered")
        time.sleep(1)
        break

# Wait until both buggers are full/data is acquired
fillState = False

while not fillState:
    fillState = rp.rp_AcqAxiGetBufferFillState(rp.RP_CH_1)[1]
print("DMA buffer full")

# Stop the acquisition
rp.rp_AcqStop()
print("Stop DMA acq\n")

# Get write pointer on the triggering location
posChA = rp.rp_AcqAxiGetWritePointerAtTrig(rp.RP_CH_1)[1]
posChB = rp.rp_AcqAxiGetWritePointerAtTrig(rp.RP_CH_2)[1]


# Allocate memory for the data
buff1 = rp.i16Buffer(READ_DATA_SIZE)
buff2 = rp.i16Buffer(READ_DATA_SIZE)


# Writing data into a text file
with open("out.txt", "w", encoding="ascii") as fp:
    read_size = 0

    while read_size < DATA_SIZE:
        size1 = READ_DATA_SIZE
        size2 = READ_DATA_SIZE
        rp.rp_AcqAxiGetDataRaw(rp.RP_CH_1, posChA, size1, buff1.cast())
        rp.rp_AcqAxiGetDataRaw(rp.RP_CH_2, posChB, size2, buff2.cast())
        for i in range(0, READ_DATA_SIZE):
            fp.write(f"{i+1:6d}:  {buff1[i]:6d}\t{buff2[i]:6d}\n")

        posChA += size1
        posChB += size2
        read_size += READ_DATA_SIZE
        print(f"Saved data size {read_size}\n")


# Plot the last buffer
for i in range(0,READ_DATA_SIZE):
    data1[i] = buff1[i]
    data2[i] = buff2[i]
    

figure, axis = plt.subplots(2, 1)

axis[0].plot(x, data1) 
axis[0].set_title("IN1 data") 

axis[1].plot(x, data2) 
axis[1].set_title("IN2 data") 

plt.show()

### Releasing resources ###
print("\nReleasing resources\n")
rp.rp_AcqAxiEnable(rp.RP_CH_1, False)
rp.rp_AcqAxiEnable(rp.RP_CH_2, False)

rp.rp_Release()

The data is saved into a text file for later processing.

### Note
There are a lot of different commands for the Acquisition. The list of available functions is quite an achievement to read through, so from now on, please refer to the *C and Python API section* of the [Command table](https://redpitaya.readthedocs.io/en/latest/appsFeatures/remoteControl/command_list.html#list-of-supported-scpi-api-commands) for all available commands.
